# Names Gender Classification:
> inpired By http://www.nltk.org/book/ch06.html

Detecting patterns is a central part of Natural Language Processing. For example, words ending in -ed, tend to be past tense verbs.

The goal of this notebook is to answer the following question:

1. How can we identify particular features of language data that are salient for classifying it?

at the same time, during this notebook, we will see how the NLP methodology is typically applied. 



## Load the Corpus:
The first thing to do is to load the corpus.

In [1]:
def load_corpus(filename):
    samples = []
    with open(filename, "r") as file:
        for line in file:
            samples.append((line.split("\t")[1].strip(), 
                            line.split("\t")[0]))
    return samples

In [2]:
corpus = load_corpus("names.txt")

In [3]:
corpus[:10]

[('Filomena', 'F'),
 ('Rhea', 'F'),
 ('Shakai', 'M'),
 ('Romilda', 'F'),
 ('Adamarys', 'F'),
 ('Abb', 'M'),
 ('Shyenne', 'F'),
 ('Tracilynn', 'F'),
 ('Gailen', 'M'),
 ('Cashel', 'M')]

In [4]:
len(corpus)

500

## Creation of Train/Dev/Test splits:
As we have seen in the theoretical part of the course one importante step before starting our experiments is to divide our corpora in training and test sets (typically 80%–90% of the corpus is used for training and the rest for testing – the reference). If you are learning from a corpus or if you are writing rules for it, you should use the training corpus. Otherwise, when you evaluate your system you will do so with something that your learning algorithm or yourself already know, which leads to an unsound evaluation.

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
train_test_split?

In [7]:
train, test = train_test_split(corpus, test_size=0.2)
test, dev = train_test_split(test, test_size=0.5)

In [8]:
len(train), len(dev), len(test)

(400, 50, 50)

### Corpus inspection:
now that we have our train set we can should look into our data, understand it and think of ways to solve the problem at hands.

In [9]:
train[:10]

[('Patrycja', 'F'),
 ('Darel', 'M'),
 ('Faris', 'M'),
 ('Louvenia', 'F'),
 ('Rogelio', 'M'),
 ('Council', 'M'),
 ('Romilda', 'F'),
 ('Beverly', 'F'),
 ('Jarik', 'M'),
 ('Katrin', 'F')]

Lets count the number of feminine and masculine names.

In [20]:
sum([1 for name, gender in train if gender == 'F'])

228

In [21]:
sum([1 for name, gender in train if gender == 'M'])

172

## Baseline:
As we observed, the numer of feminine names is higher in our train set. For this reason, we will create a simplistic baseline that assigns the 'F' label to every sample. This will be the starting point of our system.

In [22]:
def baseline_classifier(name):
    return 'F'

In [23]:
def run_baseline(names):
    """ Function that receives a list of names and return the baseline predictions """
    predictions = []
    for name in names:
        predictions.append(baseline_classifier(name))
    return predictions

In [24]:
baseline_predictions = run_baseline(names=[name for name, gender in dev])

### Precision and Recall
Now that we have our baseline predictions we can evaluate our precison and recall for the development set.

In [25]:
true_labels = [gender for name, gender in dev]

In [26]:
from sklearn.metrics import precision_score, recall_score

**Note:** The *precision_score* and *recall_score* functions require our "labels" to be numpy arrays with numeric values. For that reason we will convert the *baseline_predictions* list and the *true_labels* list to numpy arrays.

In [27]:
import numpy as np
def labels2array(labels):
    array = np.zeros(len(labels))
    for i in range(len(array)):
        if labels[i] == 'F':
            array[i] = 1
    return array

In [28]:
baseline_predictions = labels2array(baseline_predictions)
true_labels = labels2array(true_labels)

Finally we can compute our scores:

In [34]:
precision_score(true_labels, baseline_predictions, average='binary')

0.56

In [37]:
recall_score(true_labels, baseline_predictions, average='binary')

1.0

## Exercise 1:
Inspect the recall_score function and explain why the recall is 1.

In [38]:
## write your answer here.

## Our system:
Now that we know the performance of our simplistic baseline we can try to improve upon that with a classifier that uses manually crafted features to compute the probability of a given label.

**Note:** If you still don't know about classifiers don't worry. Just think of it as a "black box" that improves the final classification if your "features" are informative enough.

In [40]:
from nltk import NaiveBayesClassifier

def get_gender_features(name):
    name = name.lower()
    return {
        'last_char': name[-1],
        'first': name[0],
        'length': len(name)
    }

Extracting features for our train and development set.

In [53]:
train_features = [(get_gender_features(name), gender) for (name, gender) in train]
dev_features = [(get_gender_features(name), gender) for (name, gender) in dev]

In [54]:
train_features[0]

({'first': 'p', 'last_char': 'a', 'length': 8}, 'F')

With those features we can now train our Naive Bayes classifier.

In [55]:
classifier = NaiveBayesClassifier.train(train_features)

In [58]:
predictions = labels2array(classifier.classify_many([features for features, label in dev_features]))

In [61]:
precision_score(true_labels, predictions, average='binary')

0.75

In [62]:
recall_score(true_labels, predictions, average='binary')

0.75

With only 3 features we were able to beat our simplistic baseline.

### Exercise 2:
Go back to the *get_gender_features* function and try to improve the results by adding more features.

## Informative features:

One really cool function from the nltk classifiers is the *show_most_informative_features*.
With this function we can examine the classifier to determine which features it found most effective for distinguishing the names' genders:

In [65]:
classifier.show_most_informative_features(5)

Most Informative Features
               last_char = 'o'                 M : F      =     15.2 : 1.0
               last_char = 'a'                 F : M      =     10.5 : 1.0
               last_char = 'd'                 M : F      =      7.1 : 1.0
               last_char = 's'                 M : F      =      4.5 : 1.0
                   first = 'p'                 M : F      =      4.4 : 1.0


This listing shows that the names in the training set that end in "a" are female 33 times more often than they are male, but names that end in "k" are male 32 times more often than they are female. These ratios are known as likelihood ratios, and can be useful for comparing different feature-outcome relationships.

### Exercise 3:
Now that we now what are the most relevant features, can we build a classifier based only on rules? What is the precision and recall of such classifier?

In [ ]:
### Run you code here

## Running the final test:
Finally, after running several experiments with our dev set we should choose our best experiments and access the final performance in the test set.

In [ ]:
## Run your code here